<a href="https://colab.research.google.com/github/t3noch/colabs/blob/main/Copia_de_09_1_test_time_compute_mejorado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 📌 1️⃣ Instalar YOLOv8 y Tesseract en Colab
!pip install ultralytics opencv-python pytesseract
!apt-get install -y tesseract-ocr

In [ ]:
# 📌 2️⃣ Importar librerías necesarias
import cv2
import numpy as np
import pytesseract
import matplotlib.pyplot as plt
from google.colab import files
from ___ import YOLO

In [ ]:
# 📌 3️⃣ Configurar Tesseract en Colab
pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"

# 📌 4️⃣ Cargar un modelo YOLO preentrenado para detección de placas
modelo_yolo = YOLO("yolov8n.pt")  # Usa el modelo base preentrenado

In [ ]:
# 📌 5️⃣ Subir una imagen desde la computadora
uploaded = files.upload()
image_path = list(uploaded.keys())[0]  # Nombre del archivo subido

# 📌 6️⃣ Cargar la imagen de la matrícula
imagen = cv2.imread(___)
imagen_rgb = ___.cvtColor(imagen, cv2.COLOR_BGR2RGB)  # Convertir para visualización en Matplotlib

In [ ]:
# 📌 7️⃣ Función para aplicar Test-Time Augmentation (TTA)
def aplicar_tta(img):
    transformaciones = []

    # Imagen original
    transformaciones.append(img)

    # Ajuste de brillo y contraste
    bright = cv2.convertScaleAbs(img, alpha=1.2, beta=20)
    transformaciones.append(bright)

    # Corrección de perspectiva (simulada con rotación)
    rows, cols = img.shape[:2]
    M = cv2.getRotationMatrix2D((cols/2, rows/2), 10, 1)
    rotada = cv2.warpAffine(img, M, (cols, rows))
    transformaciones.append(rotada)

    # Reducción de ruido con desenfoque
    blur = cv2.GaussianBlur(img, (5, 5), 0)
    transformaciones.append(blur)

    return transformaciones

# 📌 8️⃣ Función para extraer texto usando Tesseract OCR
def leer_placa(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_thresh = cv2.adaptiveThreshold(img_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    texto = pytesseract.image_to_string(img, config='--psm 8')
    print('Numeros y letras reconocidos:', texto.strip())
    return texto.strip()



In [ ]:
# 📌 9️⃣ Detectar objetos en la imagen con YOLO
print("\n🔍 Detectando matrículas en la imagen...")
resultados = modelo_yolo(imagen)

# 📌 🔟 Buscar la detección de una matrícula
placa_detectada = False
predicciones = []

for resultado in resultados:
    """print("Deteccion de Yolo")
    x1, y1, x2, y2 = map(int, box.xyxy[0])  # Coordenadas de la caja detectada
    recorte_placa = imagen[y1:y2, x1:x2]
    plt.imshow(recorte_placa)
    plt.axis("off")
    plt.show()"""
    for box in resultado.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Coordenadas de la caja detectada
        recorte_placa = imagen[y1:y2, x1:x2]  # Extraer la matrícula

        # 📌 🔟 Aplicar TTA (Test-Time Augmentation) a la matrícula detectada
        imagenes_modificadas = aplicar_tta(recorte_placa)

        # 📌 1️⃣1️⃣ Extraer texto usando OCR en cada versión de la imagen
        predicciones_tta = [leer_placa(img) for img in imagenes_modificadas]

        # 📌 1️⃣2️⃣ Filtrar predicciones (descartar resultados con baja confianza)
        predicciones_filtradas = [p for p in predicciones_tta if len(p) >= 5]

        if predicciones_filtradas:
            # 📌 1️⃣3️⃣ Contar ocurrencias de cada predicción
            conteo_predicciones = {p: predicciones_filtradas.count(p) for p in set(predicciones_filtradas)}

            # 📌 1️⃣4️⃣ Verificar si alguna predicción aparece al menos 2 veces
            coincidencias = [placa for placa, frecuencia in conteo_predicciones.items() if frecuencia >= 2]

            if coincidencias:
                # Si hay al menos una predicción con 2 coincidencias, tomar la más frecuente
                placa_final = max(coincidencias, key=conteo_predicciones.get)
            else:
                placa_final = "NO RECONOCIDA"
        else:
            placa_final = "NO RECONOCIDA"

        # 📌 1️⃣3️⃣ Mostrar la imagen con la detección y el recorte
        plt.figure(figsize=(10, 5))

        # Imagen con detección
        plt.subplot(1, 2, 1)
        cv2.rectangle(imagen_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Dibujar rectángulo
        plt.imshow(imagen_rgb)
        plt.title("Detección de Matrícula")
        plt.axis("off")

        # Recorte de la matrícula
        plt.subplot(1, 2, 2)
        plt.imshow(cv2.cvtColor(recorte_placa, cv2.COLOR_BGR2RGB))
        plt.title(f"Predicción Final: {placa_final}")
        plt.axis("off")

        plt.show()

        print(f"\n✅ Matrícula detectada (TTC aplicado): {placa_final}")
        placa_detectada = True
        #break  # Solo mostrar la primera detección

# 📌 1️⃣4️⃣ Si no se detectó matrícula, mostrar mensaje
if not placa_detectada:
    print("\n⚠️ No se detectó ninguna matrícula en la imagen. Prueba con otra imagen.")